# Live Camera Feed

## From Theory to Practice

In the previous lessons, you learned how light becomes intelligence:
1. **Photons** hit the camera sensor
2. **Photodiodes** convert light to electrical signals
3. **Digital pixels** represent the image as numbers
4. **Neural networks** process patterns in the pixel data
5. **AI outputs** structured understanding of what it sees

Now you'll see this process happening in real-time. Your camera is capturing 30 frames per second, and while this preview shows the raw images, the IMX500 sensor is simultaneously running AI processing on every frame.

## What You're Observing

The live feed below demonstrates several key concepts:

**Real-time Processing**: Images are captured, processed, and displayed continuously

**Digital Image Formation**: Each frame contains hundreds of thousands of pixels with RGB color values

**System Integration**: Camera hardware, software drivers, and display work together seamlessly

**Edge Computing**: All processing happens locally on your Raspberry Pi - no internet required

While you're seeing the visual output, remember that your IMX500 sensor is simultaneously analyzing each frame for objects, calculating confidence scores, and determining bounding box coordinates - just like we discussed in the previous lessons.

## Live Camera Preview

Run the cell below to start your live camera feed:

In [ ]:
import subprocess
import cv2
import numpy as np
from IPython.display import display, clear_output, Image
import ipywidgets as widgets
import threading
import time

# Stop button in notebook
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop Preview',
    button_style='danger',
    icon='square'
)

display(stopButton)

def camera_preview():
    # Use libcamera-vid to stream MJPEG into stdout
    cmd = [
        "libcamera-vid",
        "--inline",           # Needed for streaming
        "-t", "0",            # No timeout
        "--width", "640",
        "--height", "480",
        "--codec", "mjpeg",
        "-o", "-"             # Output to stdout
    ]
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.DEVNULL)

    data = b""
    display_handle = display(None, display_id=True)

    while not stopButton.value:
        chunk = proc.stdout.read(1024)
        if not chunk:
            break
        data += chunk
        a = data.find(b'\xff\xd8')  # JPEG start
        b = data.find(b'\xff\xd9')  # JPEG end
        if a != -1 and b != -1:
            jpg = data[a:b+2]
            data = data[b+2:]
            frame = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            _, jpeg = cv2.imencode('.jpg', frame)
            display_handle.update(Image(data=jpeg.tobytes()))
    proc.terminate()

# Start preview in a separate thread
print("It takes 15-30 Seconds for the camera feed to begin the first time.")
threading.Thread(target=camera_preview, daemon=True).start()

## Understanding What Just Happened

### Technical Process
1. **libcamera-vid** captures raw image data from the IMX500 sensor
2. **MJPEG codec** compresses each frame for efficient transmission
3. **OpenCV** decodes the compressed images back into pixel arrays
4. **Jupyter display** shows the processed frames in real-time

### AI Processing Reality
While you see the visual feed, remember that simultaneously:
- The IMX500's built-in neural network is analyzing every frame
- Object detection algorithms are running at 30+ FPS
- Confidence scores and bounding boxes are being calculated
- All processing happens in under 33 milliseconds per frame

### System Verification
If your camera feed is working smoothly, your system is ready for AI applications. You've verified:
- ✅ Camera hardware is functioning
- ✅ Software drivers are properly installed
- ✅ Image processing pipeline works correctly
- ✅ Real-time performance meets requirements

## Connecting to Previous Learning

This live feed demonstrates the concepts from lessons 211 and 212:

**From 211 - Introduction to AI**:
- Light photons are being captured and converted to digital pixels
- The sensor processes this data through its neural network
- What you see visually is just the raw image; the AI "sees" objects and patterns

**From 212 - AI Camera Systems**:
- This is Edge AI in action - processing happens locally on the sensor
- Real-time performance (30 FPS) demonstrates the IMX500's capabilities
- Each frame could produce object detections with confidence scores and locations

## What's Next?

Now that your camera system is verified and working, you're ready to move beyond raw video to actual AI detection. In the next lesson, we'll add the AI layer to see object detection results overlaid on the live video feed.

**Next notebook**: AI Object Detection in Real-Time - where you'll see bounding boxes, confidence scores, and object labels appear automatically over your live video.